# Pyspark: Logistic Regression

In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

24/08/07 14:31:44 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 14:31:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 14:31:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 14:31:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/07 14:31:46 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/07 14:31:46 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

24/08/07 14:33:40 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
my_data.show() # labels are binary classification 0 1

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [6]:
my_log_reg_model = LogisticRegression()

In [9]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [10]:
log_summary = fitted_logreg.summary

In [12]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [14]:
lr_train, lr_test = my_data.randomSplit([0.7,0.3])

In [15]:
final_model = LogisticRegression()

In [16]:
fit_final = final_model.fit(lr_train)

In [17]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [18]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[123,124,125...|[37.4061725455774...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[37.5882131463740...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[17.2406556792699...|[0.99999996745544...|       0.0|
|  0.0|(692,[124,125,126...|[25.0152921747887...|[0.99999999998632...|       0.0|
|  0.0|(692,[126,127,128...|[34.9574588641983...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[19.6220346465405...|[0.99999999699213...|       0.0|
|  0.0|(692,[128,129,130...|[25.5322663509940...|[0.99999999999184...|       0.0|
|  0.0|(692,[129,130,131...|[18.2534048765929...|[0.99999998817919...|       0.0|
|  0.0|(692,[150,151,152...|[22.8604734497304...|[0.99999999988201...|       0.0|
|  0.0|(692,[151

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [20]:
my_eval = BinaryClassificationEvaluator()

In [21]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [23]:
my_final_roc # the are under ROC plot is 1 when the data is highly seperable

1.0

## for binary classification we use ROC area under the curve or percision recall area under the curve as an evaluator
## for multiclass classification we use accuracy, percision, recall as an evaluator